In [4]:
# =========================
# Cell 1 — Config
# =========================
# EITHER: point to Task-1 engineered file (recommended: faster, consistent)
USE_CLEAN_FIRST = True
CLEAN_PATH = "loan_clean_subset.csv"   # upload this if you have it

# OR: fall back to raw file (we'll do a minimal preprocess if clean is absent)
RAW_PATH = "accepted_2007_to_2018Q4.csv"  # upload if needed
NROWS_FROM_RAW = 200_000    # None for full data (Colab can handle; start smaller if RAM is tight)

RANDOM_STATE = 42


In [1]:
# =========================
# Cell 2 — Install & Imports (Fixed)
# =========================
!pip install -U d3rlpy==2.4.0
!pip install -U gymnasium[classic-control]==0.29.1

import os, gc, textwrap, numpy as np, pandas as pd, matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

import d3rlpy
from d3rlpy.dataset import MDPDataset
from d3rlpy.algos import CQL



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
C:\Users\kamya\anaconda3\envs\rl-fintech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# =========================
# Cell 3 — Helpers (same mapping as your previous tasks)
# =========================
DEFAULT_LIKE = {
    "Charged Off","Default",
    "Late (31-120 days)","Late (16-30 days)",
    "Does not meet the credit policy. Status:Charged Off"
}
PAID_LIKE = {"Fully Paid","Does not meet the credit policy. Status:Fully Paid"}

def map_target(status: str):
    if pd.isna(status): return np.nan
    s = str(status).strip()
    if s in PAID_LIKE: return 0
    if s in DEFAULT_LIKE: return 1
    return np.nan

def parse_pct(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series.astype(str).str.rstrip("%"), errors="coerce")

def parse_emp_length(series: pd.Series) -> pd.Series:
    def _map(v):
        if pd.isna(v): return np.nan
        s = str(v).strip().lower()
        if s in ("n/a","na","none"): return np.nan
        if s.startswith("<"): return 0.5
        if "10+" in s: return 10.0
        for tok in s.split():
            try: return float(tok)
            except: pass
        return np.nan
    return series.apply(_map)

FEATURES_RAW = [
    "loan_amnt","funded_amnt","term","installment","int_rate",
    "annual_inc","dti","emp_length","home_ownership","verification_status",
    "purpose","addr_state","revol_bal","revol_util",
    "open_acc","total_acc","delinq_2yrs","inq_last_6mths","pub_rec",
    "issue_d","earliest_cr_line","loan_status"
]

def add_date_features(frame: pd.DataFrame) -> pd.DataFrame:
    X = frame.copy()
    if "issue_d" in X:
        X["issue_year"] = X["issue_d"].dt.year
        X["issue_month"] = X["issue_d"].dt.month
        X["issue_ym"] = X["issue_year"]*12 + X["issue_month"]
    if "earliest_cr_line" in X:
        if "issue_d" in X:
            X["credit_hist_months"] = (
                (X["issue_d"].dt.year - X["earliest_cr_line"].dt.year)*12 +
                (X["issue_d"].dt.month - X["earliest_cr_line"].dt.month)
            )
        else:
            ref = pd.Timestamp("2018-12-01")
            X["credit_hist_months"] = (
                (ref.year - X["earliest_cr_line"].dt.year)*12 +
                (ref.month - X["earliest_cr_line"].dt.month)
            )
    for d in ("issue_d","earliest_cr_line"):
        if d in X: X.drop(columns=[d], inplace=True)
    return X

def robust_time_split(mdf: pd.DataFrame):
    # 80/20 by issue_d if feasible; else stratified random
    if "issue_d" in mdf.columns and mdf["issue_d"].notna().mean() >= 0.7:
        temp = mdf[mdf["issue_d"].notna()].copy()
        cutoff = temp["issue_d"].quantile(0.80)
        train_idx = temp.index[temp["issue_d"] <= cutoff]
        test_idx  = temp.index[temp["issue_d"] >  cutoff]
        train_df = mdf.loc[train_idx].copy()
        test_df  = mdf.loc[test_idx].copy()
        # Attach NaT rows to train to avoid empties (ok for this task)
        nat_rows = mdf.index[mdf["issue_d"].isna()]
        train_df = pd.concat([train_df, mdf.loc[nat_rows]], axis=0)
        if len(train_df)>0 and len(test_df)>0 and train_df["default"].nunique()==2:
            return train_df, test_df
    # Fallback
    tr, te = train_test_split(mdf, test_size=0.2, random_state=42, stratify=mdf["default"])
    return tr.copy(), te.copy()


In [5]:
# =========================
# Cell 4 — Load engineered CLEAN file if present; else minimally preprocess RAW
# =========================
df_clean = None
if USE_CLEAN_FIRST and os.path.exists(CLEAN_PATH):
    print("[Info] Using clean engineered file:", CLEAN_PATH)
    df_clean = pd.read_csv(CLEAN_PATH)
    assert "default" in df_clean.columns, "Clean file must contain 'default' column."

if df_clean is None:
    print("[Info] CLEAN not found → preprocessing raw:", RAW_PATH)
    hdr = pd.read_csv(RAW_PATH, compression="infer", nrows=0, low_memory=True)
    usecols = [c for c in FEATURES_RAW if c in hdr.columns]
    df = pd.read_csv(RAW_PATH, compression="infer", usecols=usecols, nrows=NROWS_FROM_RAW, low_memory=True)
    print("Raw shape:", df.shape)

    if "int_rate" in df:   df["int_rate"] = parse_pct(df["int_rate"])
    if "revol_util" in df: df["revol_util"] = parse_pct(df["revol_util"])
    if "emp_length" in df: df["emp_length"] = parse_emp_length(df["emp_length"])
    for dcol in ("issue_d","earliest_cr_line"):
        if dcol in df: df[dcol] = pd.to_datetime(df[dcol], format="%b-%Y", errors="coerce")

    df["default"] = df["loan_status"].apply(map_target)
    df = df[~df["default"].isna()].copy()

    # Build modeling frame
    keep_cols = [c for c in df.columns if c != "loan_status"]
    model_df = df[keep_cols].copy()
    train_df, test_df = robust_time_split(model_df)

    X_train_raw = train_df.drop(columns=["default"])
    X_test_raw  = test_df.drop(columns=["default"])
    y_train = train_df["default"].astype(int).values
    y_test  = test_df["default"].astype(int).values

    X_train = add_date_features(X_train_raw)
    X_test  = add_date_features(X_test_raw)

else:
    print("Clean file shape:", df_clean.shape)
    X = df_clean.drop(columns=["default"])
    y = df_clean["default"].astype(int).values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

print("Train rows:", len(y_train), " Test rows:", len(y_test))
display(X_train.head())


[Info] Using clean engineered file: loan_clean_subset.csv
Clean file shape: (177016, 24)
Train rows: 141612  Test rows: 35404


,loan_amnt,funded_amnt,term,installment,int_rate,annual_inc,dti,emp_length,home_ownership,verification_status,...,revol_util,open_acc,total_acc,delinq_2yrs,inq_last_6mths,pub_rec,issue_year,issue_month,issue_ym,credit_hist_months
151954,10000.0,10000.0,60 months,232.64,13.99,25813.22,25.89,9.0,OWN,Verified,...,34.9,10.0,19.0,2.0,0.0,0.0,2015,10,24190,587
127528,35000.0,35000.0,60 months,808.42,13.67,114000.00,17.28,10.0,MORTGAGE,Verified,...,82.0,13.0,24.0,0.0,2.0,0.0,2015,12,24192,261
107844,7275.0,7275.0,36 months,250.95,14.65,130000.00,5.35,10.0,RENT,Source Verified,...,34.7,5.0,13.0,0.0,0.0,0.0,2015,8,24188,119
164311,24000.0,24000.0,36 months,722.76,5.32,108000.00,9.44,3.0,MORTGAGE,Not Verified,...,18.9,13.0,36.0,0.0,0.0,0.0,2015,12,24192,403
59881,22000.0,22000.0,60 months,478.23,10.99,95000.00,10.13,6.0,MORTGAGE,Source Verified,...,82.7,6.0,11.0,0.0,0.0,0.0,2015,10,24190,110


In [6]:
# =========================
# Cell 5 — Preprocess to numeric arrays (impute, OHE, scale)
# =========================
num_features = [c for c in X_train.columns if pd.api.types.is_numeric_dtype(X_train[c])]
cat_features = [c for c in X_train.columns if not pd.api.types.is_numeric_dtype(X_train[c])]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocess = ColumnTransformer([
    ("num", numeric_transformer, num_features),
    ("cat", categorical_transformer, cat_features),
])

X_train_np = preprocess.fit_transform(X_train)
X_test_np  = preprocess.transform(X_test)

print("Final feature dim:", X_train_np.shape[1])


Final feature dim: 88


In [8]:
# =========================
# Cell 6 — Build Offline RL dataset (Contextual bandit augmentation)
# =========================
# We create one-step transitions for BOTH actions per state:
# - action=1 (Approve) → reward depends on outcome
# - action=0 (Deny)    → reward = 0
#
# NOTE: This introduces counterfactual entries for action=0. This is acceptable for this task
# to let the agent compare approve vs deny. In production, prefer logged bandit data with IPS/DR.

def compute_reward(approve: int, default_flag: int, loan_amnt: float, int_rate: float) -> float:
    if approve == 0:
        return 0.0
    # approve == 1
    if default_flag == 0:  # fully paid
        return float(loan_amnt) * float(int_rate) / 100.0  # int_rate is a percent value
    else:  # defaulted
        return -float(loan_amnt)

# For rewards we need loan_amnt & int_rate; pull them from ORIGINAL (preprocessed) frames.
# If using CLEAN file, those columns should be present. If not, you can add them to the clean pipeline.
assert "loan_amnt" in X_train.columns and "int_rate" in X_train.columns, \
    "loan_amnt and int_rate must be present in features for the reward function."

# Build transitions for TRAIN
obs_train = []
act_train = []
rew_train = []
ter_train = []

for i in range(len(y_train)):
    s = X_train_np[i]
    la = X_train.iloc[i]["loan_amnt"]
    ir = X_train.iloc[i]["int_rate"]
    d  = int(y_train[i])

    # action 0 (deny)
    obs_train.append(s); act_train.append(0)
    rew_train.append(compute_reward(0, d, la, ir)); ter_train.append(1.0)

    # action 1 (approve)
    obs_train.append(s); act_train.append(1)
    rew_train.append(compute_reward(1, d, la, ir)); ter_train.append(1.0)

obs_train = np.asarray(obs_train, dtype=np.float32)
act_train = np.asarray(act_train, dtype=np.int64)
rew_train = np.asarray(rew_train, dtype=np.float32)
ter_train = np.asarray(ter_train, dtype=np.float32)

train_dataset = MDPDataset(
    observations=obs_train,
    actions=act_train,
    rewards=rew_train,
    terminals=ter_train,
)

# Build transitions for TEST (for evaluation simulation)
obs_test = []
act_test = []
rew_test = []
ter_test = []

for i in range(len(y_test)):
    s = X_test_np[i]
    la = X_test.iloc[i]["loan_amnt"]
    ir = X_test.iloc[i]["int_rate"]
    d  = int(y_test[i])

    # Same augmentation to allow greedy eval
    obs_test.append(s); act_test.append(0)
    rew_test.append(compute_reward(0, d, la, ir)); ter_test.append(1.0)

    obs_test.append(s); act_test.append(1)
    rew_test.append(compute_reward(1, d, la, ir)); ter_test.append(1.0)

obs_test = np.asarray(obs_test, dtype=np.float32)
act_test = np.asarray(act_test, dtype=np.int64)
rew_test = np.asarray(rew_test, dtype=np.float32)
ter_test = np.asarray(ter_test, dtype=np.float32)

test_dataset = MDPDataset(
    observations=obs_test,
    actions=act_test,
    rewards=rew_test,
    terminals=ter_test,
)

# --- replace the two print lines in Cell 6 with this block ---

n_train = obs_train.shape[0]
n_test  = obs_test.shape[0]

print(f"Train transitions: {n_train:,}")
print(f"Test transitions : {n_test:,}")

# Each state was duplicated with actions {0,1}, so unique states = transitions / 2
print(f"Unique train states (applications): {n_train // 2:,}")
print(f"Unique test  states (applications): {n_test  // 2:,}")

# Optional extra sanity checks
print("obs_train shape:", obs_train.shape, " actions:", act_train.shape,
      " rewards:", rew_train.shape, " terminals:", ter_train.shape)
print("obs_test  shape:", obs_test.shape,  " actions:", act_test.shape,
      " rewards:", rew_test.shape,  " terminals:", ter_test.shape)

# If you’re curious whether your d3rlpy version exposes a size method:
try:
    print("train_dataset.size():", train_dataset.size())
    print("test_dataset.size():",  test_dataset.size())
except Exception:
    pass  # not all versions expose .size()



2025-10-29 16:40.23 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(88,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-10-29 16:40.23 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-29 16:40.25 [info     ] Action size has been automatically determined. action_size=2
2025-10-29 16:40.30 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(88,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-10-29 16:40.30 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-29 16:40.30 [info     ] Action size has been automatically determined. action_size=2
Train tr

In [14]:
# ---- Robust trainer that works across d3rlpy versions ----
def robust_fit(algo, dataset, logdir="./logs_cql", verbose=True):
    attempts = []

    # Most common modern signatures
    attempts.append(lambda: algo.fit(dataset, n_epochs=20, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, epochs=20, logdir=logdir, verbose=verbose))

    # Positional-only epochs (older versions)
    attempts.append(lambda: algo.fit(dataset, 20, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, 20))

    # Step-based training (some versions prefer total gradient steps)
    attempts.append(lambda: algo.fit(dataset, n_steps=100_000, logdir=logdir, verbose=verbose))
    attempts.append(lambda: algo.fit(dataset, total_steps=100_000, logdir=logdir, verbose=verbose))

    # Last resort: minimal call
    attempts.append(lambda: algo.fit(dataset))

    last_err = None
    for i, call in enumerate(attempts, 1):
        try:
            print(f"[robust_fit] Trying signature #{i} ...")
            return call()
        except TypeError as e:
            print(f"[robust_fit] Signature #{i} failed: {e}")
            last_err = e
        except Exception as e:
            print(f"[robust_fit] Signature #{i} failed (other): {e}")
            last_err = e
    raise last_err if last_err else RuntimeError("All fit() signatures failed.")

# ---- Call the robust trainer ----
robust_fit(algo, train_dataset, logdir="./logs_cql", verbose=True)


[robust_fit] Trying signature #1 ...
[robust_fit] Signature #1 failed: QLearningAlgoBase.fit() got an unexpected keyword argument 'n_epochs'
[robust_fit] Trying signature #2 ...
[robust_fit] Signature #2 failed: QLearningAlgoBase.fit() got an unexpected keyword argument 'epochs'
[robust_fit] Trying signature #3 ...
[robust_fit] Signature #3 failed: QLearningAlgoBase.fit() got an unexpected keyword argument 'logdir'
[robust_fit] Trying signature #4 ...
2025-10-29 16:44.58 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(88,)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-29 16:44.58 [info     ] Directory is created at d3rlpy_logs\DiscreteCQL_20251029164458
2025-10-29 16:44.58 [debug    ] Building models...            
2025-10-29 16:44.59 [debug    ] Mod

[]

In [15]:
# =========================
# Cell 8 — Evaluate policy offline on TEST set
# =========================
# Greedy action per state: a_hat = argmax_a Q(s,a)
# Then compute the realized reward using the ground-truth label and your reward function.

def greedy_action(algo: CQL, states: np.ndarray) -> np.ndarray:
    # d3rlpy's predict expects batch of observations
    return algo.predict(states)

# Deduplicate the paired (s, a=0) and (s, a=1) in test_dataset to one unique state list
# Since we appended actions in order [0, 1], take every other obs as the unique state.
unique_states = obs_test[::2]   # every two entries share the same state
assert unique_states.shape[0] == len(y_test)

a_hat = greedy_action(algo, unique_states)  # 0 or 1 per state

# Compute rewards under your rule using X_test original fields + y_test
rewards = []
approved_defaults = 0
approved_paid = 0

for i in range(len(y_test)):
    la = X_test.iloc[i]["loan_amnt"]
    ir = X_test.iloc[i]["int_rate"]
    d  = int(y_test[i])
    ah = int(a_hat[i])

    r = compute_reward(ah, d, la, ir)
    rewards.append(r)

    if ah == 1:
        if d == 1: approved_defaults += 1
        else:      approved_paid += 1

rewards = np.array(rewards, dtype=float)

avg_reward = rewards.mean()
approval_rate = float((a_hat == 1).mean())

print(f"Avg reward per application: {avg_reward:,.2f}")
print(f"Approval rate: {approval_rate*100:.2f}%")
print(f"Approved & Fully Paid count: {approved_paid}")
print(f"Approved & Defaulted count : {approved_defaults}")

# Simple confusion-style table for approvals on test:
# rows = model decision (Deny/Approve), cols = outcome (Paid/Default)
deny_paid = ((a_hat == 0) & (y_test == 0)).sum()
deny_def  = ((a_hat == 0) & (y_test == 1)).sum()
app_paid  = ((a_hat == 1) & (y_test == 0)).sum()
app_def   = ((a_hat == 1) & (y_test == 1)).sum()

print("\nDecision vs Outcome (test):")
print(pd.DataFrame(
    [[deny_paid, deny_def],
     [app_paid,  app_def]],
    index=["Deny","Approve"],
    columns=["Paid","Default"]
))


Avg reward per application: -165.57
Approval rate: 9.79%
Approved & Fully Paid count: 2736
Approved & Defaulted count : 731

Decision vs Outcome (test):
          Paid  Default
Deny     25463     6474
Approve   2736      731


In [16]:
# =========================
# Cell 9 — Save artifacts (policy + preprocessor) for reuse
# =========================
ART_DIR = "/content/offline_rl_cql"
os.makedirs(ART_DIR, exist_ok=True)

# Save d3rlpy model
algo.save(os.path.join(ART_DIR, "cql_discrete_model.d3"))

# Save the sklearn preprocess pipeline (so you can serve this policy later)
import joblib
joblib.dump(preprocess, os.path.join(ART_DIR, "preprocess.joblib"))

# Save a quick report
with open(os.path.join(ART_DIR, "report.txt"), "w") as f:
    f.write(textwrap.dedent(f"""
    Offline RL — CQL (Discrete) — One-step Loan Approval
    Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}
    Avg reward per application (test): {avg_reward:,.4f}
    Approval rate (test)              : {approval_rate*100:.2f}%
    Approved & Fully Paid (count)     : {approved_paid}
    Approved & Defaulted (count)      : {approved_defaults}
    """).strip())

print("Saved:")
print(" -", os.path.join(ART_DIR, "cql_discrete_model.d3"))
print(" -", os.path.join(ART_DIR, "preprocess.joblib"))
print(" -", os.path.join(ART_DIR, "report.txt"))


Saved:
 - /content/offline_rl_cql\cql_discrete_model.d3
 - /content/offline_rl_cql\preprocess.joblib
 - /content/offline_rl_cql\report.txt
